# review data to dataframe

In [716]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np

# 1. google crawrling
## 1.1 googl main

In [604]:
#https://chromedriver.storage.googleapis.com/index.html?path=80.0.3987.106/
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")
#assert "Google" in driver.title

element = driver.find_element_by_name('q')
element.clear()
element.send_keys('google map')
element.submit()

#### 브라우저 닫기

In [420]:
#driver.close()

## 1.2. google map site 들어가기

In [605]:
element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

## 1.3. 원하는 장소 클릭
#### 검색어 입력

In [606]:
element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
element.clear()

In [607]:
element.send_keys('타임스퀘어')
element.submit()


#### bs4로 html parser

In [608]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
all_address =soup.find_all('div', attrs = {'class': "suggest-left-cell"} )

real_address = '영등포동4가'
element_index=0
for i,address in enumerate(all_address):
#     print(address.contents[0], type(address.contents[0]))
    if real_address in address.text:
        element_index=i+1 
        break
        
print(element_index)

3


#### 주소 맞는 곳 클릭

In [609]:
# element = driver.find_element_by_xpath("//ul[@class='sbsb_b']/li[2]")
element = driver.find_element_by_xpath(f"//ul[@class='sbsb_b']/li[{element_index}]")
element.click()

## 1.4. 주소 가져오기

In [610]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

In [611]:
address =soup.find_all('span',attrs = {'class': "widget-pane-link"} )
address[2].text

'서울특별시 영등포구 영등포동 영중로 15'

## 1.5. review 가져오기

In [612]:
element = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
#element = driver.find_element_by_xpath('//span[@class="allxGeDnJMl__text gm2-button-alt"]')

element.click()

#### 스크롤 다운
- https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

In [616]:
element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
element.click()

In [617]:
SCROLL_PAUSE_TIME = 2

In [618]:
while  True:
    html = driver.find_element_by_tag_name('html')
    text1 = html.text
    html.send_keys(Keys.END)
    time.sleep(SCROLL_PAUSE_TIME)
    
    html = driver.find_element_by_tag_name('html')
    text2 = html.text

    if text1 == text2:
        break
    else:
        continue

In [171]:
html = driver.page_source

#### bs4로 html parser

In [172]:
soup = BeautifulSoup(html, 'html.parser')

In [173]:
total_score =soup.find_all('div',attrs = {'class': "gm2-display-2"} )
total_score



[<div class="gm2-display-2" jsan="7.gm2-display-2" jstcache="964" style='font-family: "Google Sans Display", "Google Sans", Roboto, "Noto Sans KR", Arial, sans-serif;'>3.7</div>]

In [174]:
total_score[0].string

'3.7'

#### 개별 리뷰

In [175]:
total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )
print(len(total_review))

2


In [176]:
review=[]
for i, review in enumerate(total_review):
    print(i, review.text)
    review.append(review.text)
print(len(review))

0 쾌적한 사무공간
1 깔끔하고 주차편하고....
2


#### 개별 별점

In [177]:
scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )


In [178]:
score_list = []
for score in scores:
    print(score.attrs['aria-label'])
    score_list.append(score.attrs['aria-label'])
    
print(len(score_list))

 별표 4개 
 별표 4개 
2


# 2. 함수로 만들기

## 2.1. chrome 브라우저 열기

In [448]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")
#assert "Google" in driver.title


time.sleep(2)


element = driver.find_element_by_name('q')


#element = WebDriverWait(driver, 10).until(lambda x: x.driver.find_element_by_name('q'))
        

#delay = 10
#try:
#    element=WebDriverWait(driver, delay).until(EC.presence_of_element_located(driver.find_element_by_name('q')))
#except TimeoutException:
#    print ("Loading took too much time!-Try again")


element.clear()
element.send_keys('google map')
element.submit()


#google map site 들어가기
element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

## 2.2. 샵 선택

In [674]:
def get_addScore(html,shop_address):

    soup = BeautifulSoup(html, 'html.parser')

    infos =soup.find_all('span',attrs = {'class': "section-info-text"} )
    for i, info in enumerate(infos):
        if shop_address in info:
            address = info.text
    address =infos[0].text
    
    total_score = soup.find_all('div',attrs = {'class': "gm2-display-2"} )

    if len(total_score)!=0:
        total_score = total_score[0].text
    else:
        total_score = None
    
    return address, total_score
    

In [700]:
def search_shop(shop_name, shop_address):
    SCROLL_PAUSE_TIME = 2

    # shop 이름 검색
    element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
    element.clear()

    element.send_keys(shop_name)
    element.submit()

    time.sleep(2)
    
    # 일치하는 shop 주소 찾기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    all_address =soup.find_all('div', attrs = {'class': "suggest"} )

    element_index=1
    for i,address in enumerate(all_address):
    #     print(address.contents[0], type(address.contents[0]))
        if shop_address in address.text:
            element_index=i+1 
            break

    print('element_index',element_index)
    
    try:
        select_shop = driver.find_element_by_xpath(f"//ul[@class='sbsb_b']/li[{element_index}]")
    except: 
        select_shop = driver.find_element_by_xpath(f"//div[@class='suggestions']/div[@class='sbsb_b']/div[{element_index}]")    
    finally:
        select_shop.click()
    
    
    time.sleep(2)
    
    # shop 주소, 총 별점 갖고오기
    html = driver.page_source
    address, total_score =get_addScore(html,shop_address)
    
    if total_score == None:
        review_list = None
        score_list= None
    else:
        time.sleep(2)

        # 전체 리뷰 페이지 보기 : https://m.blog.naver.com/kiddwannabe/221310063239
        # 스크롤 조절
        review_text = driver.find_element_by_xpath('//div[@class="jqnFjrOWMVU__right"]')
        driver.execute_script("arguments[0].scrollIntoView(true);", review_text);


        # 버튼 클릭
        review_button = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
        review_button.click()


        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤 다운
        try:
            element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
            element.click()

        except Exception as err:
            print(err)
       
        i = 0
        while  True:
            i += 1
            html = driver.find_element_by_tag_name('html')
            time.sleep(SCROLL_PAUSE_TIME)

            text1 = html.text
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.find_element_by_tag_name('html')
            text2 = html.text

            if text1 == text2:
                break
            else:
                continue

        # ELEMENT 가 화면에 보이도록 스크롤 조정 --> 아래 부분 추가로 확장됨


        # 개별 리뷰, 별점 갖고오기
        html = driver.page_source

        review_list, score_list=get_review(html)

    return address, total_score,review_list, score_list
    
 

In [701]:
def get_review(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 개별 리뷰
    total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )

    review_list=[]
    for i, review in enumerate(total_review):
        review_list.append(review.text)

    #### 개별 별점
    scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )

    score_list = []
    for score in scores:
        score_list.append(score.attrs['aria-label'])

    return review_list, score_list

# 3. dataframe

## 2.1 total_table

In [734]:
total_table= pd.DataFrame(columns = ['ID', '이름', '주소', 'total_score'])
total_table

,ID,이름,주소,total_score


In [735]:
ID = '123'

In [736]:
total_table.loc[1]=[ID, shop_name, address, total_score]

In [737]:
total_table

,ID,이름,주소,total_score
1,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0


In [772]:
def total_table_insert(total_table, ID, shop_name, address, total_score):
    print(len(total_table))
    if total_score != None:
        total_table.loc[len(total_table)+1] = [ID, shop_name, address, total_score]
    else:
        total_table.loc[len(total_table)+1] = [ID, shop_name, address,np.nan]
    return total_table

In [773]:
total_table_insert(total_table,ID, shop_name, address, total_score)

7


,ID,이름,주소,total_score
1,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0
2,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0
3,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0
4,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0
5,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0
6,123,우리들 동물병원,서울특별시 노원구 중계2.3동,4.0
7,123,사랑 동물병원,서울특별시 강북구 우이동 319-12,None
8,123,사랑 동물병원,서울특별시 강북구 우이동 319-12,NaN


## 2.2. review table

In [722]:
review_table= pd.DataFrame(columns = ['ID', 'num', 'review', 'score'])


In [726]:
for i, (review, score) in enumerate(zip(review_list, score_list)):
    review_table.loc[i]=[ID, i, review, score]

In [727]:
review_table

,ID,num,riview,score
0,123,0,친절함 비용도 저렴한거같음!!\n단점은 여선생님은 정말 친절하시고 좋은데\n남성분은...,별표 5개
1,123,1,수술을 잘하시고...왠만한 검사는 안하심..ㅋ\n그게 과하지 않아 좋을수도 있으나 ...,별표 4개
2,123,2,여러 병원 다니다 주변분 소개로 갔는데 정말 친절하신 설명 감사하네요. 단 주차가 ...,별표 5개
3,123,3,불 친절하고 너무 성의가 없어요.\n\n《수술비도... 카드로 할 경우 \n현금보다...,별표 1개
4,123,4,완전 최고 의사선생님이 과잉진료없이 진료비 과대청구없이 성실히 진료해주세요 구로구에...,별표 5개


In [815]:
def review_table_insert(review_table, ID, shop_name, review_list, score_list):
    print(len(review_table))
    if review_list == None:
        review_table.loc[1+len(review_table)] = [ID, 1, np.nan, np.nan ]
    else:
        for i, (review, score) in enumerate(zip(review_list, score_list)):
            review_table.loc[i+1+len(review_table)] = [ID, i, review, score]   
    
    return review_table

In [811]:
review_table_insert(review_table, ID, shop_name, review_list, score_list)

21


,ID,num,riview,score
0,123,0,친절함 비용도 저렴한거같음!!\n단점은 여선생님은 정말 친절하시고 좋은데\n남성분은...,별표 5개
1,123,1,수술을 잘하시고...왠만한 검사는 안하심..ㅋ\n그게 과하지 않아 좋을수도 있으나 ...,별표 4개
2,123,2,여러 병원 다니다 주변분 소개로 갔는데 정말 친절하신 설명 감사하네요. 단 주차가 ...,별표 5개
3,123,3,불 친절하고 너무 성의가 없어요.\n\n《수술비도... 카드로 할 경우 \n현금보다...,별표 1개
4,123,4,완전 최고 의사선생님이 과잉진료없이 진료비 과대청구없이 성실히 진료해주세요 구로구에...,별표 5개
5,123,0,친절함 비용도 저렴한거같음!!\n단점은 여선생님은 정말 친절하시고 좋은데\n남성분은...,별표 5개
7,123,1,수술을 잘하시고...왠만한 검사는 안하심..ㅋ\n그게 과하지 않아 좋을수도 있으나 ...,별표 4개
9,123,2,여러 병원 다니다 주변분 소개로 갔는데 정말 친절하신 설명 감사하네요. 단 주차가 ...,별표 5개
11,123,3,불 친절하고 너무 성의가 없어요.\n\n《수술비도... 카드로 할 경우 \n현금보다...,별표 1개
13,123,4,완전 최고 의사선생님이 과잉진료없이 진료비 과대청구없이 성실히 진료해주세요 구로구에...,별표 5개


# 4.실행

In [816]:
shop_name = '사랑 동물병원'
shop_address = '당산'
ID = '123'

address, total_score,review_list, score_list=search_shop(shop_name,shop_address)
print('address',address)
print('total_score',total_score)
try:
    print(len(review_list), len(score_list))
except:
    pass
print('review_list',review_list)
print('score_list',score_list)

try:
    back_button = driver.find_element_by_xpath('//button[@class="ozj7Vb3wnYq__action-button-clickable"]')
    back_button.click()
except:
    pass

total_table_insert(total_table,ID, shop_name, address, total_score)
review_table_insert(review_table, ID, shop_name, review_list, score_list)
print(len(total_table), len(review_table))

element_index 1
address 서울특별시 강북구 우이동 319-12
total_score None
review_list None
score_list None
17
21
18 22


In [817]:
total_table.tail()

,ID,이름,주소,total_score
14,123,행복 동물병원,경기도 광주시 곤지암읍 삼리 6-1,4.0
15,123,해피 동물병원,경기도 용인시 기흥구 보정동 700-6,3.0
16,123,행복한 동물병원,경기도 성남시 분당구 삼평동 722,3.0
17,123,행복한 동물병원,경기도 성남시 분당구 삼평동 722,3.0
18,123,사랑 동물병원,서울특별시 강북구 우이동 319-12,NaN


In [819]:
review_table.tail()

,ID,num,riview,score
19,123,2,여러 병원 다니다 주변분 소개로 갔는데 정말 친절하신 설명 감사하네요. 단 주차가 ...,별표 5개
21,123,0,집가까이 있어서 좋아요,별표 3개
23,123,4,완전 최고 의사선생님이 과잉진료없이 진료비 과대청구없이 성실히 진료해주세요 구로구에...,별표 5개
20,123,0,오랜 경험을 갖춘 진료와수술 전문의 수의사가있고 미용사도있고 최신식 의료기기및 수술...,별표 4개
22,123,1,NaN,NaN


> x-path 조절하기
- http://blog.naver.com/PostView.nhn?blogId=kiddwannabe&logNo=221323738780&categoryNo=35&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

> 스크롤 조절
https://m.blog.naver.com/kiddwannabe/221310063239